<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/11GirosYAgregaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introducción a la ciencia de datos con Python
### Rafa Caballero

### Giros y agrupaciones
Operaciones muy útiles para preparar los dataframes antes de empezar a trabajar

### Índice
[Trasponer](#Trasponer)<br>
[pivot](#Pivot)<br>
[melt](#Melt)<br>
[pivot_table](#Pivot_table)<br>
[Agregaciones](#Agregaciones)<br>

<a name="Trasponer"></a>
## Trasponer

Tenemos datos del euro en excel y queremos hacer una gráfica de su evolución

In [ ]:
#!pip install --upgrade xlrd # si da error la siguiente línea descomentar esto y ejecutar

In [ ]:
url = "https://github.com/RafaelCaballero/tdm/raw/master/datos/euro.xlsx"
import pandas as pd

df = pd.read_excel(url)
df

Eliminamos las 3 últimas filas:


In [ ]:
df2 = df.iloc[:-3,:]
df2

Ahora tenemos un problema, queremos hacer una gráfica de PX_OPEN pero es una fila. Solución: trasponer

In [ ]:
df3 = df2.T
df3

In [ ]:
df4 = df3.copy()
df4.columns = df3.loc["Dates"]
df4

In [ ]:
df5 = df4.iloc[1:,:]
df5

In [ ]:
df5["PX_OPEN"].plot()

<a name="Pivot"></a>
## Pivot

3 parámetros relevantes

- columns: la columna (o columnas) cuyos valores serán las nuevas columnas
- values: columna con la que se rellenaran estos valores
- index: columna cuyo contenido pasará a ser el índice

<img src=https://pandas.pydata.org/docs/_images/reshaping_pivot.png width=400>

Tenemos datos de paro por comunidades:

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/parocomunidades.csv"
df = pd.read_csv(url,encoding="latin1")
df.Comunidad = df.Comunidad.str.strip() # para quitar blancos al final y al principio

Nos gustaría hacer una gráfica para ver la evolución del paro por años en alguna comunidad concreta, pero para eso deberían ser columnas

In [ ]:
df.T

No vale! no nos quedan los periodos como filas; lo que necesitamos es convertir Comunidad en columna sin afectar a las demás. Para esto podemos usar [pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html)

In [ ]:
df2 = df.pivot(index='Periodo', columns='Comunidad', values='Total')
df2

In [ ]:
df2["Asturias, Principado de"].plot()

**Pregunta** ¿Qué hará el código siguiente?

In [ ]:
#df.pivot(index='Comunidad', columns='Periodo', values='Total')

<a name="Melt"></a>
## Melt

[melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) es la función contraria, la que pasa del formato ancho al formato largo

En esta ocasión se indican las columnas que no hay que tocar, las que hacen de "clave primaria":

<img src="https://miro.medium.com/max/1400/0*O3SGf68-vlON_H8L.png" width=400>

In [ ]:
df2

In [ ]:
df2.melt()

In [ ]:
df2.melt(ignore_index=False)

<a name="Pivot_table"></a>
## pivot_table

[pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) Añade una función de agregación a pivot.

In [ ]:
stocks = pd.read_csv('https://gist.githubusercontent.com/alexdebrie/b3f40efc3dd7664df5a20f5eee85e854/raw/ee3e6feccba2464cbbc2e185fb17961c53d2a7f5/stocks.csv')
stocks

Vamos a convertir la columna `symbol`en 3, una para cada uno de sus valores

In [ ]:
df_symbol = stocks.pivot(index='date', columns='symbol', values='volume')
df_symbol

In [ ]:
df_symbol.columns

In [ ]:
stocks.pivot(index='symbol', columns='date', values='volume')

Total de volumen para cada valor

In [ ]:
import numpy as np
stocks.pivot_table(index='symbol', values='volume', aggfunc=np.sum)

**Ejemplo** Consideramos el fichero

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/raw/master/datos/clientes.xlsx"
df = pd.read_excel(url)
df

Queremos el total de tasas por cliente desglosadas en dos columnas, una para entregado=1 y otra para entregado=0

<a name="Agregaciones"></a>
## Agregados

Ya sabemos que se pueden utilizar funciones de agregación por columnas

<table>
<thead><tr><th>Function</th><th>Description</th></tr></thead><tbody><tr><td>count</td><td>Number of non-null observations</td></tr><tr><td>sum</td><td>Sum of values</td></tr><tr><td>mean</td><td>Mean of values</td></tr><tr><td>mad</td><td>Mean absolute deviation</td></tr><tr><td>median</td><td>Arithmetic median of values</td></tr><tr><td>min</td><td>Minimum</td></tr><tr><td>max</td><td>Maximum</td></tr><tr><td>mode</td><td>Mode</td></tr><tr><td>abs</td><td>Absolute Value</td></tr><tr><td>prod</td><td>Product of values</td></tr><tr><td>std</td><td>Unbiased standard deviation</td></tr><tr><td>var</td><td>Unbiased variance</td></tr><tr><td>sem</td><td>Unbiased standard error of the mean</td></tr><tr><td>skew</td><td>Unbiased skewness (3rd moment)</td></tr><tr><td>kurt</td><td>Unbiased kurtosis (4th moment)</td></tr><tr><td>quantile</td><td>Sample quantile (value at %)</td></tr><tr><td>cumsum</td><td>Cumulative sum</td></tr><tr><td>cumprod</td><td>Cumulative product</td></tr><tr><td>cummax</td><td>Cumulative maximum</td></tr><tr><td>cummin</td><td>Cumulative minimum</td></tr></tbody>
</table>



In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/valores.csv"
df = pd.read_csv(url)
df

In [ ]:
df.INDU_Index_High.max(), df.INDU_Index_High.min()

Sin embargo en otras ocasiones nos hará falta calcular estos datos por grupos. El siguiente ejemplo tiene datos de tweets de las elecciones USA2020 para varios usuarios, incluyendo el "sentimiento" que expresa el tweet sobre el candidato Biden (B) y Trump (T)

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/blob/master/datos/todousa2020_users.xlsx?raw=true"

df = pd.read_excel(url)
df

Para cada `userid` queremos saber la opinión media de estos usuarios para los dos candidatos (B) y (T).

Esto se hace en dos fases:

- Primero se agrupa con group by

In [ ]:
df_userid = df.groupby('userid')
df_userid



Ahora se hacen las operaciones que se deseen sobre los grupos creados

In [ ]:
df_user = df_userid.agg(
      t_media=('T', 'mean'),
      b_media=('B', 'mean')
)
df_user

**Ejercicio** Ahora queremos saber en el fichero

https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/vote_intention.csv

que contiene datos de encuestas, la edad media de los votantes de cada partido (el voto es vote_r y la edad age)

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/vote_intention.csv"
df = pd.read_csv(url)
df

In [ ]:
import numpy as np
df.pivot_table(index='vote_r', values='age', aggfunc=np.mean).sort_values(by=["age"])


Otra forma:

In [ ]:
df_vote_r = df.groupby('vote_r')
df_vote_r.mean()[["age"]].sort_values(by=["age"])

Mejor ¿por qué?

In [ ]:
df_vote_r = df[['vote_r',"age"]].groupby('vote_r')
df_vote_r.mean().sort_values(by=["age"])

¿Y si se quiere sobre dos variables?

In [ ]:
df.pivot_table(index='vote_r', values=['age',"edu"], aggfunc=np.mean).sort_values(by=["edu"])

In [ ]:
df_vote_r = df[['vote_r',"age","edu"]].groupby('vote_r')
df_vote_r.mean().sort_values(by=["edu"])

Obtención de grupos

In [ ]:
grupos = df_vote_r = df[['vote_r',"age"]].groupby('vote_r')
grupos.get_group("Nulo")